# Solving Einstein equation to get Kottler solution

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://relativite.obspm.fr/blackholes/).
It involves computations based on differential geometry tools developed through the [SageManifolds project](https://sagemanifolds.obspm.fr).

In [1]:
version()

'SageMath version 9.5.beta5, Release Date: 2021-10-28'

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [2]:
%display latex

## Spacetime

We declare the spacetime manifold $M$:

In [3]:
M = Manifold(4, 'M')
print(M)

4-dimensional differentiable manifold M


We declare the chart of spherical coordinates $(t,r,\theta,\phi)$:

In [4]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

The static and spherically symmetric metric ansatz, with the unknown functions $A(r)$ and $B(r)$:

In [5]:
g = M.lorentzian_metric('g')
A = function('A')
B = function('B')
g[0,0] = -A(r)
g[1,1] = B(r)
g[2,2] = r^2
g[3,3] = (r*sin(th))^2
g.display()

g = -A(r) dt⊗dt + B(r) dr⊗dr + r^2 dth⊗dth + r^2*sin(th)^2 dph⊗dph

The Christoffel symbols of $g$, with respect to the default chart:

In [6]:
g.christoffel_symbols_display()

Gam^t_t,r = 1/2*d(A)/dr/A(r) 
Gam^r_t,t = 1/2*d(A)/dr/B(r) 
Gam^r_r,r = 1/2*d(B)/dr/B(r) 
Gam^r_th,th = -r/B(r) 
Gam^r_ph,ph = -r*sin(th)^2/B(r) 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

## Einstein equation

The cosmological constant:

In [7]:
Lamb = var('Lamb', latex_name=r'\Lambda')
Lamb

Lamb

The Einstein equation:

In [8]:
EE = g.ricci() - 1/2*g.ricci_scalar()*g + Lamb*g
EE.set_name('E')
print(EE)

Field of symmetric bilinear forms E on the 4-dimensional differentiable manifold M


In [9]:
EE.display_comp()

E_t,t = -((Lamb*r^2 - 1)*A(r)*B(r)^2 - r*A(r)*d(B)/dr + A(r)*B(r))/(r^2*B(r)^2) 
E_r,r = ((Lamb*r^2 - 1)*A(r)*B(r) + r*d(A)/dr + A(r))/(r^2*A(r)) 
E_th,th = 1/4*(4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr)/(A(r)^2*B(r)^2) 
E_ph,ph = 1/4*(4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr)*sin(th)^2/(A(r)^2*B(r)^2)

### Simplifying and rearranging the equations

In [10]:
eq0 = EE[0,0]*r^2*B(r)^2/A(r); eq0

-((Lamb*r^2 - 1)*A(r)*B(r)^2 - r*A(r)*d(B)/dr + A(r)*B(r))/A(r)

In [11]:
eq1 = EE[1,1]*r^2*A(r); eq1

(Lamb*r^2 - 1)*A(r)*B(r) + r*d(A)/dr + A(r)

In [12]:
eq2 = EE[2,2]*4*A(r)^2*B(r)^2; eq2

4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr

In [13]:
eq3 = EE[3,3]*4*A(r)^2*B(r)^2/sin(th)^2; eq3

4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr

In [14]:
eq3 == eq2

True

## Solving Einstein equation

The following combination of eq0 and eq1 is particularly simple:

In [15]:
eq4 = (eq0*A(r) + eq1*B(r))/r; eq4

(r*B(r)*d(A)/dr + r*A(r)*d(B)/dr)/r

The solution is $A(r)B(r)=C$, where $C$ is a constant:

In [16]:
s = desolve(eq4.expr() == 0, B(r), ivar=r)
s

_C/A(r)

Let us rename the constant to $\alpha$:

In [17]:
var('alpha', latex_name=r'\alpha')
B_sol(r) = s.subs(_C=alpha); B_sol

r |--> alpha/A(r)

We replace $B(r)$ by the above value in the remaining equations:

In [18]:
eq5 = X.function(eq1.expr().substitute_function(B, B_sol)); eq5

(Lamb*r^2 - 1)*alpha + r*d(A)/dr + A(r)

In [19]:
eq6 = X.function(eq2.expr().substitute_function(B, B_sol)); eq6

4*Lamb*alpha^2*r^2 - alpha*r^2*(d(A)/dr)^2/A(r) + 2*alpha*r^2*d^2(A)/dr^2 + 2*alpha*r*d(A)/dr + (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*alpha*d(A)/dr/A(r)^2

Let us solve `eq5` for $A(r)$. Note that we are using `eq5.expr()` to get a symbolic expression, as expected by the function `desolve`, while `eq5` is a coordinate function. 

In [20]:
s = desolve(eq5.expr() == 0, A(r), ivar=r)
s.expand()

-1/3*Lamb*alpha*r^2 + alpha + _C/r

We rename the constant $C$ to $-2m$ and set the value of constant $\alpha$ to $1$:

In [21]:
var('m')
A_sol(r) = s.subs(_C=-2*m, alpha=1).expand()
A_sol

r |--> -1/3*Lamb*r^2 - 2*m/r + 1

Let us check that `eq6` is fulfilled by the found value of $A(r)$:

In [22]:
eq6.expr().substitute_function(A, A_sol).subs(alpha=1).simplify_full()

0

### Final expression of the metric

We have got the Kottler metric:

In [23]:
g[0,0] = -A_sol(r)
g[1,1] = 1/A_sol(r)
g.display()

g = (1/3*Lamb*r^2 + 2*m/r - 1) dt⊗dt - 3/(Lamb*r^2 + 6*m/r - 3) dr⊗dr + r^2 dth⊗dth + r^2*sin(th)^2 dph⊗dph

which reduces to Schwarzschild metric as soon as the cosmological constant vanishes.

In [24]:
g.christoffel_symbols_display()

Gam^t_t,r = (Lamb*r^3 - 3*m)/(Lamb*r^4 + 6*m*r - 3*r^2) 
Gam^r_t,t = 1/9*(Lamb^2*r^6 + 3*Lamb*m*r^3 - 3*Lamb*r^4 - 18*m^2 + 9*m*r)/r^3 
Gam^r_r,r = -(Lamb*r^3 - 3*m)/(Lamb*r^4 + 6*m*r - 3*r^2) 
Gam^r_th,th = 1/3*Lamb*r^3 + 2*m - r 
Gam^r_ph,ph = 1/3*(Lamb*r^3 + 6*m - 3*r)*sin(th)^2 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

Let us check that the vacuum Einstein equation is satisfied by the above metric:

In [25]:
EE = g.ricci() - 1/2*g.ricci_scalar()*g + Lamb*g
EE.set_name('E')
EE.display()

E = 0

The Ricci scalar is constant for this solution:

In [26]:
g.ricci_scalar().display()

r(g): M → ℝ
   (t, r, th, ph) ↦ 4*Lamb

The Ricci tensor is proportional to the metric tensor:

In [27]:
g.ricci().display()

Ric(g) = 1/3*(Lamb^2*r^3 + 6*Lamb*m - 3*Lamb*r)/r dt⊗dt - 3*Lamb*r/(Lamb*r^3 + 6*m - 3*r) dr⊗dr + Lamb*r^2 dth⊗dth + Lamb*r^2*sin(th)^2 dph⊗dph

In [28]:
g.ricci() == Lamb * g

True